# 1- Packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
from textwrap import wrap
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


%matplotlib inline

# 2 - Data acquisition

In [44]:
Europe=pd.read_excel(r'C:\Users\Eric.Li\Documents\IPO data\IPO.xlsx',sheet_name="Europe").set_index("Ticker")
price_Europe=pd.read_excel(r'C:\Users\Eric.Li\Documents\IPO data\IPO.xlsx',sheet_name="Europe price").set_index("Ticker")
index_Europe=pd.read_excel(r'C:\Users\Eric.Li\Documents\IPO data\IPO.xlsx',sheet_name="Europe index").set_index("Ticker")

In [59]:
def get_return(price_df,index_df):
    abs_return=price_df.diff(1,axis=1)/price_df.shift(1,axis=1)
    index_return=index_df.diff(1,axis=1)/index_df.shift(1,axis=1)
    rel_return=abs_return-index_return
    return abs_return,rel_return

In [60]:
abs_return_Europe,rel_return_Europe=get_return(price_Europe,index_Europe)

# 3 - Strategy

In [233]:
def IPO_long(data_df,price_df,return_df,Size,Open,Volume,liquidity_constraint,start_day,end_day):
    target_data=data_df[(data_df["Market_cap"]>=Size)&(data_df["Offer_Day1_Open"]>Open)&(data_df["Volume_shares"]>Volume)&(data_df["5d_liquidity"]>liquidity_constraint)]
    target_price=price_df.loc[target_data.index.tolist()]
    high_evaluate=(target_price.expanding(min_periods=start_day,axis=1).max()==target_price).iloc[:,start_day-1:end_day]
    high_evaluate=high_evaluate[high_evaluate==True].dropna(how='all',axis=0)
    trade_day=high_evaluate.apply(lambda x:x.tolist().index(1.0),axis=1)    
    target_return=return_df.loc[trade_day.index]
    signal_df=target_return.apply(lambda x:x[5-1+day[x.name]:5-1+day[x.name]+40],axis=1).shift(1,axis=1)
    signal_df["Index"]=signal_df.apply(lambda x:(x.name,data_df["IPO_date"].loc[x.name].date(),trade_day[x.name]+6),axis=1)
    signal_df=signal_df.set_index("Index")
    return signal_df

In [244]:
signal_df=IPO_long(Europe,price_Europe,rel_return_Europe,500,0.1,0.25,5,5,50)

KeyError: ('PDX SS Equity', 'occurred at index PDX SS Equity')